In [2]:
from rdflib import Graph, Namespace, RDF, OWL, RDFS

# Load the ontology
ontology_path = "hi_ontology.ttl"  # Update with the correct path
g = Graph()
g.parse(ontology_path, format="turtle")

# Define namespace
HI = Namespace("http://www.semanticweb.org/hi_ontology#")

# Extract all Classes
existing_classes = [str(cls) for cls in g.subjects(RDF.type, OWL.Class)]

# Extract all Subclasses (Fix: Ensure child-parent relationships are captured)
existing_subclasses = []
for child in g.subjects(RDFS.subClassOf, None):
    for parent in g.objects(subject=child, predicate=RDFS.subClassOf):
        existing_subclasses.append((str(child), str(parent)))

# Extract all Object Properties
existing_object_properties = [str(prop) for prop in g.subjects(RDF.type, OWL.ObjectProperty)]

# Extract all Data Properties
existing_data_properties = [str(prop) for prop in g.subjects(RDF.type, OWL.DatatypeProperty)]

# Extract all Research Papers (Instances of ResearchPaper class)
existing_papers = [str(paper) for paper in g.subjects(RDF.type, HI["ResearchPaper"])]

# Print Summary
print("\n✅ **Existing Classes:**", len(existing_classes))
print(existing_classes)

print("\n✅ **Existing Subclasses:**", len(existing_subclasses))
print(existing_subclasses)

print("\n✅ **Existing Object Properties:**", len(existing_object_properties))
print(existing_object_properties)

print("\n✅ **Existing Data Properties:**", len(existing_data_properties))
print(existing_data_properties)

print("\n✅ **Existing Research Papers:**", len(existing_papers))
print(existing_papers)


✅ **Existing Classes:** 15
['http://www.semanticweb.org/vbr240/ontologies/2022/4/untitled-ontology-51/Context', 'http://www.semanticweb.org/vbr240/ontologies/2022/4/untitled-ontology-51/Domain', 'http://www.semanticweb.org/vbr240/ontologies/2022/4/untitled-ontology-51/Endgoal', 'http://www.semanticweb.org/vbr240/ontologies/2022/4/untitled-ontology-51/EthicalConsideration', 'http://www.semanticweb.org/vbr240/ontologies/2022/4/untitled-ontology-51/ArtificialAgent', 'http://www.semanticweb.org/vbr240/ontologies/2022/4/untitled-ontology-51/InformationProcessing', 'http://www.semanticweb.org/vbr240/ontologies/2022/4/untitled-ontology-51/Human', 'http://www.semanticweb.org/vbr240/ontologies/2022/4/untitled-ontology-51/Interaction', 'http://www.semanticweb.org/vbr240/ontologies/2022/4/untitled-ontology-51/InteractionTask', 'http://www.semanticweb.org/vbr240/ontologies/2022/4/untitled-ontology-51/ProcessingMethod', 'http://www.semanticweb.org/vbr240/ontologies/2022/4/untitled-ontology-51/Proc

In [3]:
from rdflib import Graph, Namespace, RDF, OWL, RDFS, XSD, Literal

# Load the ontology
ontology_path = "hi_ontology.ttl"  # Update with the correct path
g = Graph()
g.parse(ontology_path, format="turtle")

# Define namespace
HI = Namespace("http://www.semanticweb.org/hi_ontology#")
g.bind("hi", HI)

# 🛠 Define Required Data Properties
data_properties = {
    "hasTitle": XSD.string,
    "hasAuthor": XSD.string,
    "hasYear": XSD.integer,
    "hasDOI": XSD.string,
    "hasAbstract": XSD.string,
    "hasVenue": XSD.string,
    "hasKeywords": XSD.string,
    "hasExternalLink": XSD.string
}

# 🛠 Add Data Properties (if they don’t exist)
for prop, dtype in data_properties.items():
    prop_uri = HI[prop]
    if (prop_uri, RDF.type, OWL.DatatypeProperty) not in g:
        g.add((prop_uri, RDF.type, OWL.DatatypeProperty))
        g.add((prop_uri, RDFS.domain, HI["ResearchPaper"]))  # Set domain to ResearchPaper
        g.add((prop_uri, RDFS.range, dtype))
        g.add((prop_uri, RDFS.label, Literal(prop)))
        g.add((prop_uri, RDFS.comment, Literal(f"Automatically added data property: {prop}")))

# Save the updated ontology
updated_ontology_path = "updated_hi_ontology.ttl"
g.serialize(destination=updated_ontology_path, format="turtle")

print(f"✅ Ontology updated successfully! Saved as {updated_ontology_path}")

✅ Ontology updated successfully! Saved as updated_hi_ontology.ttl


In [5]:
import requests
import time
import pdfplumber
import urllib.parse
import spacy
from rdflib import Graph, Namespace, RDF, RDFS, OWL, XSD, Literal

# 📌 Load NLP Model
nlp = spacy.load("en_core_web_sm")

# 📌 Load Ontology
ontology_path = "updated_hi_ontology.ttl"
g = Graph()
g.parse(ontology_path, format="turtle")

# 📌 Define Namespace
HI = Namespace("http://www.semanticweb.org/hi_ontology#")
g.bind("hi", HI)

# 📌 Ensure ResearchPaper Class Exists
ResearchPaper = HI["ResearchPaper"]
if (ResearchPaper, RDF.type, OWL.Class) not in g:
    g.add((ResearchPaper, RDF.type, OWL.Class))
    g.add((ResearchPaper, RDFS.label, Literal("Research Paper")))
    g.add((ResearchPaper, RDFS.comment, Literal("Class representing research papers in the ontology.")))

# 📌 Extract Metadata from CrossRef API using DOIs
def get_metadata_from_doi(doi):
    url = f"https://api.crossref.org/works/{doi}"
    response = requests.get(url).json()
    if "message" in response:
        metadata = response["message"]
        return {
            "title": metadata.get("title", ["Unknown"])[0],
            "authors": ", ".join([author.get("given", "") + " " + author.get("family", "") 
                                  for author in metadata.get("author", [])]),
            "year": metadata.get("published-print", {}).get("date-parts", [[None]])[0][0],
            "venue": metadata.get("container-title", ["Unknown"])[0],
            "doi": metadata.get("DOI", ""),
            "url": metadata.get("URL", ""),
            "abstract": metadata.get("abstract", "No abstract available")
        }
    return None

# 📌 Fetch Metadata using Semantic Scholar API (Backup Method)
API_URL = "https://api.semanticscholar.org/graph/v1/paper/search"
def get_paper_metadata(title):
    params = {
        "query": title,
        "fields": "title,authors,year,doi,venue,abstract"
    }
    response = requests.get(API_URL, params=params)
    if response.status_code == 200:
        data = response.json()
        if "data" in data and len(data["data"]) > 0:
            paper = data["data"][0]  
            return {
                "title": paper.get("title", "N/A"),
                "authors": ", ".join([author["name"] for author in paper.get("authors", [])]),
                "year": paper.get("year", "N/A"),
                "doi": paper.get("doi", "N/A"),
                "venue": paper.get("venue", "N/A"),
                "abstract": paper.get("abstract", "N/A")
            }
    else:
        print(f"❌ Error fetching metadata for: {title}")
    return None

# 📌 Extract Keywords & Topics Using TF-IDF & NLP
def extract_keywords(text):
    doc = nlp(text)
    return list(set([ent.text for ent in doc.ents if ent.label_ in {"ORG", "PRODUCT", "EVENT"}]))

# 📌 Define DOI List
doi_list = [
    "10.3233/FAIA230074", "10.3233/FAIA230085", "10.3233/FAIA240185",
    "10.3233/FAIA230098", "10.3233/FAIA220192", "10.3233/FAIA230078",
    "10.3233/FAIA220200", "10.3233/FAIA240204", "", "10.3233/FAIA220209",
    "10.3233/FAIA240195"
]

# 📌 Extract Metadata
papers_metadata = [get_metadata_from_doi(doi) for doi in doi_list]

# 📌 Extract Keywords Using NLP
for paper in papers_metadata:
    if paper and "abstract" in paper:
        paper["keywords"] = extract_keywords(paper["abstract"])
    else:
        paper["keywords"] = "No keywords available"

# 📌 Print Extracted Metadata
print("📄 Extracted Research Papers Metadata:")
for paper in papers_metadata:
    print(f"📌 {paper['title']} ({paper['year']})")
    print(f"   📝 Authors: {paper['authors']}")
    print(f"   📖 Venue: {paper['venue']}")
    print(f"   🔗 DOI: {paper['doi']}")
    print(f"   🌐 URL: {paper['url']}")
    print(f"   📑 Keywords: {paper['keywords']}\n")

# 📌 Save Extracted Metadata for Next Step
metadata_path = "extracted_metadata.json"
import json
with open(metadata_path, "w") as f:
    json.dump(papers_metadata, f, indent=4)

print(f"✅ Extracted metadata saved in {metadata_path}")

📄 Extracted Research Papers Metadata:
📌 On the Interdependence of Reliance Behavior and Accuracy in AI-Assisted Decision-Making (2023)
   📝 Authors: Jakob Schoeffer, Johannes Jakubik, Michael Voessing, Niklas Kuehl, Gerhard Satzger
   📖 Venue: Frontiers in Artificial Intelligence and Applications
   🔗 DOI: 10.3233/faia230074
   🌐 URL: https://doi.org/10.3233/faia230074
   📑 Keywords: ['AI']

📌 Value-Aware Active Learning (2023)
   📝 Authors: Burcu Sayin, Jie Yang, Andrea Passerini, Fabio Casati
   📖 Venue: Frontiers in Artificial Intelligence and Applications
   🔗 DOI: 10.3233/faia230085
   🌐 URL: https://doi.org/10.3233/faia230085
   📑 Keywords: []

📌 Human-Centered AI for Dementia Care: Using Reinforcement Learning for Personalized Interventions Support in Eating and Drinking Scenarios (2024)
   📝 Authors: Wen-Tseng Chang, Shihan Wang, Stephanie Kramer, Michel Oey, Somaya Ben Allouch
   📖 Venue: Frontiers in Artificial Intelligence and Applications
   🔗 DOI: 10.3233/faia240185
   🌐 U

In [6]:
from rdflib import Graph, Namespace, RDF, RDFS, OWL, XSD, Literal

# 📌 Load Ontology Path
ontology_path = "updated_hi_ontology.ttl"

# 📌 Load Ontology Graph
g = Graph()
g.parse(ontology_path, format="turtle")

# 📌 Define Namespace
HI = Namespace("http://www.semanticweb.org/hi_ontology#")
g.bind("hi", HI)

# 📌 Define Parent Classes (Ensure they exist)
Actor = HI["Actor"]
AIConcept = HI["AIConcept"]
ResearchPaper = HI["ResearchPaper"]

# 📌 Add Parent Classes if missing
for cls in [Actor, AIConcept, ResearchPaper]:
    if (cls, RDF.type, OWL.Class) not in g:
        g.add((cls, RDF.type, OWL.Class))

# 📌 Define New Subclasses
subclasses = {
    "Human": "Actor",
    "ArtificialAgent": "Actor",
    "Researcher": "Actor",
    "MachineLearning": "AIConcept",
    "SupervisedLearning": "MachineLearning",
    "UnsupervisedLearning": "MachineLearning",
    "ReinforcementLearning": "MachineLearning",
    "JournalPaper": "ResearchPaper",
    "ConferencePaper": "ResearchPaper",
}

# 📌 Add Subclasses to Ontology
for subclass, parent in subclasses.items():
    subclass_uri = HI[subclass]
    parent_uri = HI[parent]
    
    if (subclass_uri, RDF.type, OWL.Class) not in g:
        g.add((subclass_uri, RDF.type, OWL.Class))
        g.add((subclass_uri, RDFS.label, Literal(subclass)))
    
    g.add((subclass_uri, RDFS.subClassOf, parent_uri))

# 📌 Save Updated Ontology
g.serialize(destination=ontology_path, format="turtle")
print(f"✅ Ontology updated with subclass hierarchy! Saved as {ontology_path}")

✅ Ontology updated with subclass hierarchy! Saved as updated_hi_ontology.ttl


In [8]:
from rdflib import Graph, Namespace, RDF, RDFS, OWL

# 📌 Load the updated ontology
ontology_path = "updated_hi_ontology.ttl"
g = Graph()
g.parse(ontology_path, format="turtle")

# 📌 Define Namespace
HI = Namespace("http://www.semanticweb.org/hi_ontology#")

# 📌 Extract Classes & Subclasses
existing_classes = [str(cls) for cls in g.subjects(RDF.type, OWL.Class)]
existing_subclasses = [(str(child), str(parent)) for child, _, parent in g.triples((None, RDFS.subClassOf, None))]

# 📌 Print Class & Subclass Hierarchy
print("\n✅ **Extracted Classes:**")
for cls in existing_classes:
    print(f"  - {cls}")

print("\n✅ **Extracted Subclasses:**")
for child, parent in existing_subclasses:
    print(f"  - {child} ⊆ {parent}")

# 📌 Extract Object & Data Properties
existing_object_properties = [str(prop) for prop in g.subjects(RDF.type, OWL.ObjectProperty)]
existing_data_properties = [str(prop) for prop in g.subjects(RDF.type, OWL.DatatypeProperty)]

# 📌 Print Object & Data Properties
print("\n✅ **Extracted Object Properties:**")
for prop in existing_object_properties:
    print(f"  - {prop}")

print("\n✅ **Extracted Data Properties:**")
for prop in existing_data_properties:
    print(f"  - {prop}")

# 📌 Extract Research Papers
research_papers = [str(paper) for paper in g.subjects(RDF.type, HI["ResearchPaper"])]

print("\n✅ **Extracted Research Papers:**")
for paper in research_papers:
    print(f"  - {paper}")


✅ **Extracted Classes:**
  - http://www.semanticweb.org/hi_ontology#ArtificialAgent
  - http://www.semanticweb.org/hi_ontology#ConferencePaper
  - http://www.semanticweb.org/hi_ontology#Human
  - http://www.semanticweb.org/hi_ontology#JournalPaper
  - http://www.semanticweb.org/hi_ontology#ReinforcementLearning
  - http://www.semanticweb.org/hi_ontology#Researcher
  - http://www.semanticweb.org/hi_ontology#SupervisedLearning
  - http://www.semanticweb.org/hi_ontology#UnsupervisedLearning
  - http://www.semanticweb.org/hi_ontology#AIConcept
  - http://www.semanticweb.org/vbr240/ontologies/2022/4/untitled-ontology-51/Context
  - http://www.semanticweb.org/vbr240/ontologies/2022/4/untitled-ontology-51/Domain
  - http://www.semanticweb.org/vbr240/ontologies/2022/4/untitled-ontology-51/Endgoal
  - http://www.semanticweb.org/vbr240/ontologies/2022/4/untitled-ontology-51/EthicalConsideration
  - http://www.semanticweb.org/vbr240/ontologies/2022/4/untitled-ontology-51/ArtificialAgent
  - http

In [13]:
from rdflib import Graph, Namespace, RDF, RDFS, OWL, Literal, XSD, BNode

# Load the ontology
ontology_path = "hi_ontology.ttl"
g = Graph()
g.parse(ontology_path, format="turtle")

# Define the namespace
HI = Namespace("http://www.semanticweb.org/vbr240/ontologies/2022/4/untitled-ontology-51/")
g.bind("hi", HI)

### Step 1: Add New Subclasses ###
g.add((HI.MachineLearningSystem, RDFS.subClassOf, HI.ArtificialAgent))
g.add((HI.HumanExpert, RDFS.subClassOf, HI.Human))
g.add((HI.InteractiveRobot, RDFS.subClassOf, HI.Robot))
g.add((HI.CognitiveSystem, RDFS.subClassOf, HI.Intelligent_System))

### Step 2: Add Complex Object Properties with Restrictions ###
g.add((HI.hasProcessingTask, RDF.type, OWL.ObjectProperty))
g.add((HI.hasProcessingTask, RDFS.domain, HI.ArtificialAgent))
g.add((HI.hasProcessingTask, RDFS.range, HI.ProcessingTask))

# ✅ Define OWL Restriction on ArtificialAgent (minCardinality 1)
restriction_bnode = BNode()  # Create a blank node for restriction
g.add((restriction_bnode, RDF.type, OWL.Restriction))
g.add((restriction_bnode, OWL.onProperty, HI.hasProcessingTask))
g.add((restriction_bnode, OWL.minCardinality, Literal(1, datatype=XSD.nonNegativeInteger)))
g.add((HI.ArtificialAgent, RDFS.subClassOf, restriction_bnode))  # Attach restriction to class

### Step 3: Add Datatype Properties ###
g.add((HI.processingSpeed, RDF.type, OWL.DatatypeProperty))
g.add((HI.processingSpeed, RDFS.domain, HI.ArtificialAgent))
g.add((HI.processingSpeed, RDFS.range, XSD.float))

g.add((HI.accuracyRate, RDF.type, OWL.DatatypeProperty))
g.add((HI.accuracyRate, RDFS.domain, HI.Intelligent_System))
g.add((HI.accuracyRate, RDFS.range, XSD.decimal))

### Step 4: Add New Individuals with Properties ###
g.add((HI.DeepLearningAI, RDF.type, HI.MachineLearningSystem))
g.add((HI.DeepLearningAI, HI.processingSpeed, Literal(0.85, datatype=XSD.float)))
g.add((HI.DeepLearningAI, HI.accuracyRate, Literal(98.5, datatype=XSD.decimal)))

g.add((HI.ExpertUser, RDF.type, HI.HumanExpert))
g.add((HI.ExpertUser, HI.capability, HI.Adaptiveness))

g.add((HI.AssistantRobot, RDF.type, HI.InteractiveRobot))
g.add((HI.AssistantRobot, HI.capability, HI.Collaborativeness))

### Step 5: Save Updated Ontology ###
updated_path = "updated_hi_ontology.ttl"
g.serialize(destination=updated_path, format="turtle")

print(f"✅ Ontology updated and saved to: {updated_path}")

✅ Ontology updated and saved to: updated_hi_ontology.ttl


In [ ]:
from rdflib import Graph, Namespace, RDF, RDFS, OWL, Literal, XSD

# Load existing ontology
ontology_path = "updated_hi_ontology.ttl"
g = Graph()
g.parse(ontology_path, format="turtle")

# Define namespace
HI = Namespace("http://www.semanticweb.org/vbr240/ontologies/2022/4/untitled-ontology-51/")
g.bind("hi", HI)

### Step 1: Define New Classes (Subclasses) ###
g.add((HI.Human_Agent, RDF.type, OWL.Class))
g.add((HI.Artificial_Intelligence, RDF.type, OWL.Class))
g.add((HI.Ethical_Challenge, RDF.type, OWL.Class))

# Define subclass relationships
g.add((HI.Human_Agent, RDFS.subClassOf, HI.Actor))
g.add((HI.Artificial_Intelligence, RDFS.subClassOf, HI.ArtificialAgent))
g.add((HI.Autonomous_AI, RDFS.subClassOf, HI.Artificial_Intelligence))
g.add((HI.Ethical_AI, RDFS.subClassOf, HI.Artificial_Intelligence))
g.add((HI.Explainable_AI, RDFS.subClassOf, HI.Artificial_Intelligence))

### Step 2: Define Object Properties ###
g.add((HI.interactsWith, RDF.type, OWL.ObjectProperty))
g.add((HI.interactsWith, RDFS.domain, HI.Human_Agent))
g.add((HI.interactsWith, RDFS.range, HI.Artificial_Intelligence))

g.add((HI.hasEthicalChallenge, RDF.type, OWL.ObjectProperty))
g.add((HI.hasEthicalChallenge, RDFS.domain, HI.Artificial_Intelligence))
g.add((HI.hasEthicalChallenge, RDFS.range, HI.Ethical_Challenge))

### Step 3: Define Datatype Properties ###
g.add((HI.AI_Reliability, RDF.type, OWL.DatatypeProperty))
g.add((HI.AI_Reliability, RDFS.domain, HI.Artificial_Intelligence))
g.add((HI.AI_Reliability, RDFS.range, XSD.float))

g.add((HI.Ethical_Score, RDF.type, OWL.DatatypeProperty))
g.add((HI.Ethical_Score, RDFS.domain, HI.Artificial_Intelligence))
g.add((HI.Ethical_Score, RDFS.range, XSD.integer))

### Step 4: Define Restrictions (e.g., Min Cardinality) ###
restriction_bnode = HI.Restriction1
g.add((restriction_bnode, RDF.type, OWL.Restriction))
g.add((restriction_bnode, OWL.onProperty, HI.interactsWith))
g.add((restriction_bnode, OWL.minCardinality, Literal(1, datatype=XSD.nonNegativeInteger)))
g.add((HI.Human_Agent, RDFS.subClassOf, restriction_bnode))

### Step 5: Add Instances ###
g.add((HI.Chatbot_X, RDF.type, HI.Explainable_AI))
g.add((HI.Chatbot_X, HI.AI_Reliability, Literal(0.85, datatype=XSD.float)))
g.add((HI.Chatbot_X, HI.Ethical_Score, Literal(7, datatype=XSD.integer)))

g.add((HI.Doctor_Y, RDF.type, HI.Human_Agent))
g.add((HI.Doctor_Y, HI.interactsWith, HI.Chatbot_X))

g.add((HI.Bias_in_AI, RDF.type, HI.Ethical_Challenge))
g.add((HI.Chatbot_X, HI.hasEthicalChallenge, HI.Bias_in_AI))

### Save updated ontology ###
updated_ontology_path = "updated_hi_ontology.ttl"
g.serialize(destination=updated_ontology_path, format="turtle")

print(f"✅ Ontology updated and saved to: {updated_ontology_path}")

✅ Ontology updated and saved to: updated_hi_ontology.ttl


In [16]:
from rdflib import Graph, Namespace, Literal, RDF, RDFS, OWL, XSD, BNode

# Load the existing ontology
ontology_path = "updated_hi_ontology.ttl"
g = Graph()
g.parse(ontology_path, format="turtle")

# Define correct namespace (consistent with previous updates)
HI = Namespace("http://www.semanticweb.org/vbr240/ontologies/2022/4/untitled-ontology-51/")
g.bind("hi", HI)

# Define new Classes
new_classes = {
    "AI_Assisted_DecisionMaking": HI.AIConcept,
    "Reliance_Behavior": HI.AI_Assisted_DecisionMaking,
    "Human_AI_Interaction": HI.AIConcept,
    "TrustCalibration": HI.Reliance_Behavior,
}

# Add new classes to the ontology
for cls, parent in new_classes.items():
    g.add((HI[cls], RDF.type, OWL.Class))
    g.add((HI[cls], RDFS.subClassOf, parent))

# Define new Object Properties
new_object_properties = {
    "hasRelianceBehavior": (HI.Human_AI_Interaction, HI.Reliance_Behavior),
    "affectsDecisionMaking": (HI.Reliance_Behavior, HI.AI_Assisted_DecisionMaking),
    "requiresTrustCalibration": (HI.AI_Assisted_DecisionMaking, HI.TrustCalibration),
}

# Add Object Properties
for prop, (domain, range_) in new_object_properties.items():
    g.add((HI[prop], RDF.type, OWL.ObjectProperty))
    g.add((HI[prop], RDFS.domain, domain))
    g.add((HI[prop], RDFS.range, range_))

# Define new Datatype Properties
new_datatype_properties = {
    "trustScore": (HI.Human_AI_Interaction, XSD.float),
    "adherenceLevel": (HI.Reliance_Behavior, XSD.integer),
    "decisionAccuracy": (HI.AI_Assisted_DecisionMaking, XSD.float),
}

# Add Datatype Properties
for prop, (domain, dtype) in new_datatype_properties.items():
    g.add((HI[prop], RDF.type, OWL.DatatypeProperty))
    g.add((HI[prop], RDFS.domain, domain))
    g.add((HI[prop], RDFS.range, dtype))

# Define minimum cardinality constraint using a blank node
restriction_node = BNode()
g.add((restriction_node, RDF.type, OWL.Restriction))
g.add((restriction_node, OWL.onProperty, HI.requiresTrustCalibration))
g.add((restriction_node, OWL.minCardinality, Literal(1, datatype=XSD.nonNegativeInteger)))

# Attach the restriction to the AI_Assisted_DecisionMaking class
g.add((HI.AI_Assisted_DecisionMaking, RDFS.subClassOf, restriction_node))

# Define new Instances
new_instances = {
    "CaseStudy_TrustAI": {
        "type": HI.AI_Assisted_DecisionMaking,
        "decisionAccuracy": 0.85
    },
    "Experiment_OverReliance": {
        "type": HI.Reliance_Behavior,
        "adherenceLevel": 90
    },
    "Study_TrustCalibration": {
        "type": HI.TrustCalibration,
        "trustScore": 0.75
    }
}

# Add instances
for instance, attributes in new_instances.items():
    g.add((HI[instance], RDF.type, attributes["type"]))
    for attr, value in attributes.items():
        if attr != "type":  # Skip 'type' since it's already added
            g.add((HI[instance], HI[attr], Literal(value, datatype=XSD.float if isinstance(value, float) else XSD.integer)))

# Save the updated ontology
updated_ontology_path = "updated_hi_ontology.ttl"
g.serialize(updated_ontology_path, format="turtle")

print(f"✅ Ontology updated and saved to: {updated_ontology_path}")

✅ Ontology updated and saved to: updated_hi_ontology.ttl


In [17]:
from rdflib import Graph, Namespace, Literal, RDF, RDFS, OWL, XSD, BNode

# Load the existing ontology
ontology_path = "updated_hi_ontology.ttl"
g = Graph()
g.parse(ontology_path, format="turtle")

# Define correct namespace (keeping it consistent)
HI = Namespace("http://www.semanticweb.org/vbr240/ontologies/2022/4/untitled-ontology-51/")
g.bind("hi", HI)

# Define new Classes from the third PDF
new_classes = {
    "Clinical_AI": HI.AIConcept,
    "Trust_in_Healthcare_AI": HI.Clinical_AI,
    "AI_Deployment": HI.Clinical_AI,
    "RegulatoryCompliance": HI.AI_Deployment,
    "Human_Factors": HI.Trust_in_Healthcare_AI,
}

# Add new classes to the ontology
for cls, parent in new_classes.items():
    g.add((HI[cls], RDF.type, OWL.Class))
    g.add((HI[cls], RDFS.subClassOf, parent))

# Define new Object Properties from the third PDF
new_object_properties = {
    "requiresRegulatoryApproval": (HI.AI_Deployment, HI.RegulatoryCompliance),
    "influencesTrust": (HI.Human_Factors, HI.Trust_in_Healthcare_AI),
    "isUsedInClinicalContext": (HI.Clinical_AI, HI.AI_Deployment),
    "hasRiskFactor": (HI.Trust_in_Healthcare_AI, HI.Human_Factors),
}

# Add Object Properties
for prop, (domain, range_) in new_object_properties.items():
    g.add((HI[prop], RDF.type, OWL.ObjectProperty))
    g.add((HI[prop], RDFS.domain, domain))
    g.add((HI[prop], RDFS.range, range_))

# Define new Datatype Properties from the third PDF
new_datatype_properties = {
    "complianceScore": (HI.RegulatoryCompliance, XSD.float),
    "trustLevel": (HI.Trust_in_Healthcare_AI, XSD.float),
    "errorRate": (HI.AI_Deployment, XSD.float),
}

# Add Datatype Properties
for prop, (domain, dtype) in new_datatype_properties.items():
    g.add((HI[prop], RDF.type, OWL.DatatypeProperty))
    g.add((HI[prop], RDFS.domain, domain))
    g.add((HI[prop], RDFS.range, dtype))

# Define minimum cardinality constraints using a blank node
restriction_node = BNode()
g.add((restriction_node, RDF.type, OWL.Restriction))
g.add((restriction_node, OWL.onProperty, HI.requiresRegulatoryApproval))
g.add((restriction_node, OWL.minCardinality, Literal(1, datatype=XSD.nonNegativeInteger)))

# Attach the restriction to the AI_Deployment class
g.add((HI.AI_Deployment, RDFS.subClassOf, restriction_node))

# Define new Instances related to third PDF concepts
new_instances = {
    "FDA_Approval_Process": {
        "type": HI.RegulatoryCompliance,
        "complianceScore": 0.9
    },
    "Study_Trust_in_AI": {
        "type": HI.Trust_in_Healthcare_AI,
        "trustLevel": 0.7
    },
    "Clinical_Trial_Study": {
        "type": HI.AI_Deployment,
        "errorRate": 0.05
    }
}

# Add instances to the ontology
for instance, attributes in new_instances.items():
    g.add((HI[instance], RDF.type, attributes["type"]))
    for attr, value in attributes.items():
        if attr != "type":  # Skip 'type' since it's already added
            g.add((HI[instance], HI[attr], Literal(value, datatype=XSD.float if isinstance(value, float) else XSD.integer)))

# Save the updated ontology
updated_ontology_path = "updated_hi_ontology.ttl"
g.serialize(updated_ontology_path, format="turtle")

print(f"✅ Ontology updated and saved to: {updated_ontology_path}")

✅ Ontology updated and saved to: updated_hi_ontology.ttl


In [18]:
from rdflib import Graph, Namespace, Literal, RDF, RDFS, OWL, XSD, BNode

# Load the existing ontology
ontology_path = "updated_hi_ontology.ttl"
g = Graph()
g.parse(ontology_path, format="turtle")

# Define correct namespace (keeping it consistent)
HI = Namespace("http://www.semanticweb.org/vbr240/ontologies/2022/4/untitled-ontology-51/")
g.bind("hi", HI)

# Define new Classes from the fourth PDF
new_classes = {
    "AI_Ethics_Compliance": HI.AI_Deployment,  # Adding as a subclass under AI_Deployment
    "Explainable_AI": HI.AIConcept,
    "Bias_in_AI": HI.AI_Ethics_Compliance,
    "Fairness_Metrics": HI.Bias_in_AI,
    "Human_Trust_Factors": HI.Trust_in_Healthcare_AI,
}

# Add new classes to the ontology
for cls, parent in new_classes.items():
    g.add((HI[cls], RDF.type, OWL.Class))
    g.add((HI[cls], RDFS.subClassOf, parent))

# Define new Object Properties from the fourth PDF
new_object_properties = {
    "evaluatesBias": (HI.AI_Ethics_Compliance, HI.Bias_in_AI),
    "usesFairnessMetric": (HI.Bias_in_AI, HI.Fairness_Metrics),
    "affectsHumanTrust": (HI.Human_Trust_Factors, HI.Trust_in_Healthcare_AI),
    "requiresExplainability": (HI.AI_Deployment, HI.Explainable_AI),
}

# Add Object Properties
for prop, (domain, range_) in new_object_properties.items():
    g.add((HI[prop], RDF.type, OWL.ObjectProperty))
    g.add((HI[prop], RDFS.domain, domain))
    g.add((HI[prop], RDFS.range, range_))

# Define new Datatype Properties from the fourth PDF
new_datatype_properties = {
    "biasScore": (HI.Bias_in_AI, XSD.float),
    "fairnessIndex": (HI.Fairness_Metrics, XSD.float),
    "trustScore": (HI.Human_Trust_Factors, XSD.float),
}

# Add Datatype Properties
for prop, (domain, dtype) in new_datatype_properties.items():
    g.add((HI[prop], RDF.type, OWL.DatatypeProperty))
    g.add((HI[prop], RDFS.domain, domain))
    g.add((HI[prop], RDFS.range, dtype))

# Define minimum cardinality constraints using a blank node
restriction_node = BNode()
g.add((restriction_node, RDF.type, OWL.Restriction))
g.add((restriction_node, OWL.onProperty, HI.requiresExplainability))
g.add((restriction_node, OWL.minCardinality, Literal(1, datatype=XSD.nonNegativeInteger)))

# Attach the restriction to the AI_Deployment class
g.add((HI.AI_Deployment, RDFS.subClassOf, restriction_node))

# Define new Instances related to fourth PDF concepts
new_instances = {
    "XAI_Research_Study": {
        "type": HI.Explainable_AI,
        "requiresExplainability": True
    },
    "Fairness_Audit_Report": {
        "type": HI.Fairness_Metrics,
        "fairnessIndex": 0.85
    },
    "Bias_Detection_Tool": {
        "type": HI.Bias_in_AI,
        "biasScore": 0.12
    },
    "Trust_Survey": {
        "type": HI.Human_Trust_Factors,
        "trustScore": 0.78
    }
}

# Add instances to the ontology
for instance, attributes in new_instances.items():
    g.add((HI[instance], RDF.type, attributes["type"]))
    for attr, value in attributes.items():
        if attr != "type":  # Skip 'type' since it's already added
            g.add((HI[instance], HI[attr], Literal(value, datatype=XSD.float if isinstance(value, float) else XSD.boolean)))

# Save the updated ontology
updated_ontology_path = "updated_hi_ontology.ttl"
g.serialize(updated_ontology_path, format="turtle")

print(f"✅ Ontology updated and saved to: {updated_ontology_path}")

✅ Ontology updated and saved to: updated_hi_ontology.ttl


In [19]:
from rdflib import Graph, Namespace, Literal, RDF, RDFS, OWL, XSD, BNode

# Load the existing ontology
ontology_path = "updated_hi_ontology.ttl"
g = Graph()
g.parse(ontology_path, format="turtle")

# Define correct namespace (keeping it consistent)
HI = Namespace("http://www.semanticweb.org/vbr240/ontologies/2022/4/untitled-ontology-51/")
g.bind("hi", HI)

# Define new Classes from the fifth PDF
new_classes = {
    "Human_AI_Collaboration": HI.AIConcept,
    "Interactive_Machine_Learning": HI.Human_AI_Collaboration,
    "Trust_Calibration_Methods": HI.Human_AI_Collaboration,
    "Human_Centered_Design": HI.AI_Ethics_Compliance,
    "AI_Safety_Measures": HI.AI_Ethics_Compliance,
}

# Add new classes to the ontology
for cls, parent in new_classes.items():
    g.add((HI[cls], RDF.type, OWL.Class))
    g.add((HI[cls], RDFS.subClassOf, parent))

# Define new Object Properties from the fifth PDF
new_object_properties = {
    "involvesHumanInteraction": (HI.Human_AI_Collaboration, HI.Human),
    "requiresTrustCalibration": (HI.Human_AI_Collaboration, HI.Trust_Calibration_Methods),
    "usesInteractiveLearning": (HI.Human_AI_Collaboration, HI.Interactive_Machine_Learning),
    "followsHumanCenteredDesign": (HI.AI_Safety_Measures, HI.Human_Centered_Design),
}

# Add Object Properties
for prop, (domain, range_) in new_object_properties.items():
    g.add((HI[prop], RDF.type, OWL.ObjectProperty))
    g.add((HI[prop], RDFS.domain, domain))
    g.add((HI[prop], RDFS.range, range_))

# Define new Datatype Properties from the fifth PDF
new_datatype_properties = {
    "trustCalibrationScore": (HI.Trust_Calibration_Methods, XSD.float),
    "humanInteractionLevel": (HI.Human_AI_Collaboration, XSD.float),
    "safetyComplianceRate": (HI.AI_Safety_Measures, XSD.float),
}

# Add Datatype Properties
for prop, (domain, dtype) in new_datatype_properties.items():
    g.add((HI[prop], RDF.type, OWL.DatatypeProperty))
    g.add((HI[prop], RDFS.domain, domain))
    g.add((HI[prop], RDFS.range, dtype))

# Define minimum cardinality constraints using a blank node
restriction_node = BNode()
g.add((restriction_node, RDF.type, OWL.Restriction))
g.add((restriction_node, OWL.onProperty, HI.requiresTrustCalibration))
g.add((restriction_node, OWL.minCardinality, Literal(1, datatype=XSD.nonNegativeInteger)))

# Attach the restriction to the Human_AI_Collaboration class
g.add((HI.Human_AI_Collaboration, RDFS.subClassOf, restriction_node))

# Define new Instances related to fifth PDF concepts
new_instances = {
    "Collaborative_AI_Model": {
        "type": HI.Human_AI_Collaboration,
        "humanInteractionLevel": 0.75
    },
    "Trust_Benchmarking_Study": {
        "type": HI.Trust_Calibration_Methods,
        "trustCalibrationScore": 0.82
    },
    "AI_Ethics_Compliance_Report": {
        "type": HI.AI_Safety_Measures,
        "safetyComplianceRate": 0.91
    },
}

# Add instances to the ontology
for instance, attributes in new_instances.items():
    g.add((HI[instance], RDF.type, attributes["type"]))
    for attr, value in attributes.items():
        if attr != "type":  # Skip 'type' since it's already added
            g.add((HI[instance], HI[attr], Literal(value, datatype=XSD.float if isinstance(value, float) else XSD.boolean)))

# Save the updated ontology
updated_ontology_path = "updated_hi_ontology.ttl"
g.serialize(updated_ontology_path, format="turtle")

print(f"✅ Ontology updated and saved to: {updated_ontology_path}")

✅ Ontology updated and saved to: updated_hi_ontology.ttl


In [20]:
from rdflib import Graph, Namespace, Literal, RDF, RDFS, OWL, XSD, BNode

# Load the existing ontology
ontology_path = "updated_hi_ontology.ttl"
g = Graph()
g.parse(ontology_path, format="turtle")

# Define correct namespace (keeping it consistent)
HI = Namespace("http://www.semanticweb.org/vbr240/ontologies/2022/4/untitled-ontology-51/")
g.bind("hi", HI)

# Define new Classes from the sixth PDF
new_classes = {
    "Explainable_AI": HI.AIConcept,
    "AI_Fairness": HI.AIConcept,
    "Bias_Mitigation_Strategy": HI.AI_Fairness,
    "Causal_Inference": HI.Explainable_AI,
    "Interpretable_Models": HI.Explainable_AI,
}

# Add new classes to the ontology
for cls, parent in new_classes.items():
    g.add((HI[cls], RDF.type, OWL.Class))
    g.add((HI[cls], RDFS.subClassOf, parent))

# Define new Object Properties from the sixth PDF
new_object_properties = {
    "improvesTransparency": (HI.Explainable_AI, HI.Human),
    "hasBiasMitigation": (HI.AI_Fairness, HI.Bias_Mitigation_Strategy),
    "usesCausalInference": (HI.Explainable_AI, HI.Causal_Inference),
    "enhancesInterpretability": (HI.Explainable_AI, HI.Interpretable_Models),
}

# Add Object Properties
for prop, (domain, range_) in new_object_properties.items():
    g.add((HI[prop], RDF.type, OWL.ObjectProperty))
    g.add((HI[prop], RDFS.domain, domain))
    g.add((HI[prop], RDFS.range, range_))

# Define new Datatype Properties from the sixth PDF
new_datatype_properties = {
    "transparencyScore": (HI.Explainable_AI, XSD.float),
    "biasReductionRate": (HI.AI_Fairness, XSD.float),
    "interpretabilityLevel": (HI.Interpretable_Models, XSD.float),
}

# Add Datatype Properties
for prop, (domain, dtype) in new_datatype_properties.items():
    g.add((HI[prop], RDF.type, OWL.DatatypeProperty))
    g.add((HI[prop], RDFS.domain, domain))
    g.add((HI[prop], RDFS.range, dtype))

# Define minimum cardinality constraints using a blank node
restriction_node = BNode()
g.add((restriction_node, RDF.type, OWL.Restriction))
g.add((restriction_node, OWL.onProperty, HI.hasBiasMitigation))
g.add((restriction_node, OWL.minCardinality, Literal(1, datatype=XSD.nonNegativeInteger)))

# Attach the restriction to the AI_Fairness class
g.add((HI.AI_Fairness, RDFS.subClassOf, restriction_node))

# Define new Instances related to sixth PDF concepts
new_instances = {
    "XAI_Model": {
        "type": HI.Explainable_AI,
        "transparencyScore": 0.85
    },
    "Bias_Correction_Algorithm": {
        "type": HI.Bias_Mitigation_Strategy,
        "biasReductionRate": 0.76
    },
    "Causal_Explainability_Framework": {
        "type": HI.Causal_Inference,
        "interpretabilityLevel": 0.91
    },
}

# Add instances to the ontology
for instance, attributes in new_instances.items():
    g.add((HI[instance], RDF.type, attributes["type"]))
    for attr, value in attributes.items():
        if attr != "type":  # Skip 'type' since it's already added
            g.add((HI[instance], HI[attr], Literal(value, datatype=XSD.float if isinstance(value, float) else XSD.boolean)))

# Save the updated ontology
updated_ontology_path = "updated_hi_ontology.ttl"
g.serialize(updated_ontology_path, format="turtle")

print(f"✅ Ontology updated and saved to: {updated_ontology_path}")

✅ Ontology updated and saved to: updated_hi_ontology.ttl


In [21]:
from rdflib import Graph, Namespace, Literal, RDF, RDFS, OWL, XSD, BNode

# Load the existing ontology
ontology_path = "updated_hi_ontology.ttl"
g = Graph()
g.parse(ontology_path, format="turtle")

# Define correct namespace (keeping it consistent)
HI = Namespace("http://www.semanticweb.org/vbr240/ontologies/2022/4/untitled-ontology-51/")
g.bind("hi", HI)

# Define new Classes from the seventh PDF
new_classes = {
    "Human_in_Loop": HI.AIConcept,
    "Interactive_ML": HI.Human_in_Loop,
    "Trust_Adaptation": HI.Human_in_Loop,
    "Reinforcement_Learning": HI.AIConcept,
    "Reward_Shaping": HI.Reinforcement_Learning,
    "Interactive_Rewards": HI.Reinforcement_Learning,
}

# Add new classes to the ontology
for cls, parent in new_classes.items():
    g.add((HI[cls], RDF.type, OWL.Class))
    g.add((HI[cls], RDFS.subClassOf, parent))

# Define new Object Properties from the seventh PDF
new_object_properties = {
    "requiresHumanInput": (HI.Human_in_Loop, HI.Human),
    "adaptsTrustLevel": (HI.Trust_Adaptation, HI.AIConcept),
    "optimizesRewards": (HI.Reinforcement_Learning, HI.Reward_Shaping),
    "hasInteractiveRewards": (HI.Reward_Shaping, HI.Interactive_Rewards),
}

# Add Object Properties
for prop, (domain, range_) in new_object_properties.items():
    g.add((HI[prop], RDF.type, OWL.ObjectProperty))
    g.add((HI[prop], RDFS.domain, domain))
    g.add((HI[prop], RDFS.range, range_))

# Define new Datatype Properties from the seventh PDF
new_datatype_properties = {
    "trustLevelScore": (HI.Trust_Adaptation, XSD.float),
    "rewardOptimizationFactor": (HI.Reward_Shaping, XSD.float),
    "humanFeedbackReliability": (HI.Interactive_Rewards, XSD.float),
}

# Add Datatype Properties
for prop, (domain, dtype) in new_datatype_properties.items():
    g.add((HI[prop], RDF.type, OWL.DatatypeProperty))
    g.add((HI[prop], RDFS.domain, domain))
    g.add((HI[prop], RDFS.range, dtype))

# Define minimum cardinality constraints using a blank node
restriction_node = BNode()
g.add((restriction_node, RDF.type, OWL.Restriction))
g.add((restriction_node, OWL.onProperty, HI.requiresHumanInput))
g.add((restriction_node, OWL.minCardinality, Literal(1, datatype=XSD.nonNegativeInteger)))

# Attach the restriction to the Human_in_Loop class
g.add((HI.Human_in_Loop, RDFS.subClassOf, restriction_node))

# Define new Instances related to seventh PDF concepts
new_instances = {
    "Human_Guided_RL": {
        "type": HI.Human_in_Loop,
        "trustLevelScore": 0.78
    },
    "Adaptive_Trust_System": {
        "type": HI.Trust_Adaptation,
        "trustLevelScore": 0.85
    },
    "Reinforcement_Agent": {
        "type": HI.Reinforcement_Learning,
        "rewardOptimizationFactor": 0.92
    },
    "Interactive_Feedback_Model": {
        "type": HI.Interactive_Rewards,
        "humanFeedbackReliability": 0.88
    },
}

# Add instances to the ontology
for instance, attributes in new_instances.items():
    g.add((HI[instance], RDF.type, attributes["type"]))
    for attr, value in attributes.items():
        if attr != "type":  # Skip 'type' since it's already added
            g.add((HI[instance], HI[attr], Literal(value, datatype=XSD.float if isinstance(value, float) else XSD.boolean)))

# Save the updated ontology
updated_ontology_path = "updated_hi_ontology.ttl"
g.serialize(updated_ontology_path, format="turtle")

print(f"✅ Ontology updated and saved to: {updated_ontology_path}")

✅ Ontology updated and saved to: updated_hi_ontology.ttl


In [22]:
from rdflib import Graph, Namespace, Literal, RDF, RDFS, OWL, XSD, BNode

# Load the existing ontology
ontology_path = "updated_hi_ontology.ttl"
g = Graph()
g.parse(ontology_path, format="turtle")

# Define correct namespace (keeping it consistent)
HI = Namespace("http://www.semanticweb.org/vbr240/ontologies/2022/4/untitled-ontology-51/")
g.bind("hi", HI)

# Define new Classes from the eighth PDF
new_classes = {
    "AI_Assisted_DecisionMaking": HI.AIConcept,
    "Reliance_Behavior": HI.AI_Assisted_DecisionMaking,
    "Human_AI_Interaction": HI.AIConcept,
    "Accuracy_Assessment": HI.Human_AI_Interaction,
    "Trust_Measurement": HI.Human_AI_Interaction,
    "Decision_Support": HI.AIConcept,
}

# Add new classes to the ontology
for cls, parent in new_classes.items():
    g.add((HI[cls], RDF.type, OWL.Class))
    g.add((HI[cls], RDFS.subClassOf, parent))

# Define new Object Properties from the eighth PDF
new_object_properties = {
    "requiresTrustCalibration": (HI.AI_Assisted_DecisionMaking, HI.Trust_Measurement),
    "impactsDecisionAccuracy": (HI.Reliance_Behavior, HI.Accuracy_Assessment),
    "interactsWithHuman": (HI.Human_AI_Interaction, HI.Human),
    "supportsDecisionMaking": (HI.Decision_Support, HI.AI_Assisted_DecisionMaking),
}

# Add Object Properties
for prop, (domain, range_) in new_object_properties.items():
    g.add((HI[prop], RDF.type, OWL.ObjectProperty))
    g.add((HI[prop], RDFS.domain, domain))
    g.add((HI[prop], RDFS.range, range_))

# Define new Datatype Properties from the eighth PDF
new_datatype_properties = {
    "trustScore": (HI.Trust_Measurement, XSD.float),
    "accuracyImprovement": (HI.Accuracy_Assessment, XSD.float),
    "decisionConfidence": (HI.Decision_Support, XSD.float),
}

# Add Datatype Properties
for prop, (domain, dtype) in new_datatype_properties.items():
    g.add((HI[prop], RDF.type, OWL.DatatypeProperty))
    g.add((HI[prop], RDFS.domain, domain))
    g.add((HI[prop], RDFS.range, dtype))

# Define minimum cardinality constraints using a blank node
restriction_node = BNode()
g.add((restriction_node, RDF.type, OWL.Restriction))
g.add((restriction_node, OWL.onProperty, HI.requiresTrustCalibration))
g.add((restriction_node, OWL.minCardinality, Literal(1, datatype=XSD.nonNegativeInteger)))

# Attach the restriction to the AI_Assisted_DecisionMaking class
g.add((HI.AI_Assisted_DecisionMaking, RDFS.subClassOf, restriction_node))

# Define new Instances related to eighth PDF concepts
new_instances = {
    "Trust_Evaluation_Model": {
        "type": HI.Trust_Measurement,
        "trustScore": 0.83
    },
    "AI_Decision_Support_Tool": {
        "type": HI.Decision_Support,
        "decisionConfidence": 0.91
    },
    "Human_Reliance_Study": {
        "type": HI.Reliance_Behavior,
        "accuracyImprovement": 0.78
    },
}

# Add instances to the ontology
for instance, attributes in new_instances.items():
    g.add((HI[instance], RDF.type, attributes["type"]))
    for attr, value in attributes.items():
        if attr != "type":  # Skip 'type' since it's already added
            g.add((HI[instance], HI[attr], Literal(value, datatype=XSD.float if isinstance(value, float) else XSD.boolean)))

# Save the updated ontology
updated_ontology_path = "updated_hi_ontology.ttl"
g.serialize(updated_ontology_path, format="turtle")

print(f"✅ Ontology updated and saved to: {updated_ontology_path}")

✅ Ontology updated and saved to: updated_hi_ontology.ttl


In [23]:
from rdflib import Graph, Namespace, Literal, RDF, RDFS, OWL, XSD, BNode

# Load the existing ontology
ontology_path = "updated_hi_ontology.ttl"
g = Graph()
g.parse(ontology_path, format="turtle")

# Define correct namespace (keeping it consistent)
HI = Namespace("http://www.semanticweb.org/vbr240/ontologies/2022/4/untitled-ontology-51/")
g.bind("hi", HI)

# Define new Classes from the ninth PDF
new_classes = {
    "Clinical_AI": HI.AIConcept,
    "Trust_in_AI": HI.Clinical_AI,
    "Trust_Calibration": HI.Trust_in_AI,
    "AI_Deployment_Process": HI.Clinical_AI,
    "Healthcare_Workflow": HI.Clinical_AI,
    "Human_AI_Collaboration": HI.AIConcept,
}

# Add new classes to the ontology
for cls, parent in new_classes.items():
    g.add((HI[cls], RDF.type, OWL.Class))
    g.add((HI[cls], RDFS.subClassOf, parent))

# Define new Object Properties from the ninth PDF
new_object_properties = {
    "affectsTrust": (HI.Trust_Calibration, HI.Trust_in_AI),
    "isIntegratedIn": (HI.AI_Deployment_Process, HI.Healthcare_Workflow),
    "requiresHumanSupervision": (HI.Human_AI_Collaboration, HI.Clinical_AI),
    "involvesDecisionSupport": (HI.Human_AI_Collaboration, HI.Trust_Calibration),
}

# Add Object Properties
for prop, (domain, range_) in new_object_properties.items():
    g.add((HI[prop], RDF.type, OWL.ObjectProperty))
    g.add((HI[prop], RDFS.domain, domain))
    g.add((HI[prop], RDFS.range, range_))

# Define new Datatype Properties from the ninth PDF
new_datatype_properties = {
    "trustScore": (HI.Trust_Calibration, XSD.float),
    "workflowEfficiency": (HI.Healthcare_Workflow, XSD.float),
    "humanInvolvementLevel": (HI.Human_AI_Collaboration, XSD.float),
}

# Add Datatype Properties
for prop, (domain, dtype) in new_datatype_properties.items():
    g.add((HI[prop], RDF.type, OWL.DatatypeProperty))
    g.add((HI[prop], RDFS.domain, domain))
    g.add((HI[prop], RDFS.range, dtype))

# Define minimum cardinality constraints using a blank node
restriction_node = BNode()
g.add((restriction_node, RDF.type, OWL.Restriction))
g.add((restriction_node, OWL.onProperty, HI.requiresHumanSupervision))
g.add((restriction_node, OWL.minCardinality, Literal(1, datatype=XSD.nonNegativeInteger)))

# Attach the restriction to the Human_AI_Collaboration class
g.add((HI.Human_AI_Collaboration, RDFS.subClassOf, restriction_node))

# Define new Instances related to ninth PDF concepts
new_instances = {
    "AI_Trust_Model": {
        "type": HI.Trust_Calibration,
        "trustScore": 0.87
    },
    "Hospital_Workflow_Optimization": {
        "type": HI.Healthcare_Workflow,
        "workflowEfficiency": 0.92
    },
    "AI_Human_Team": {
        "type": HI.Human_AI_Collaboration,
        "humanInvolvementLevel": 0.78
    },
}

# Add instances to the ontology
for instance, attributes in new_instances.items():
    g.add((HI[instance], RDF.type, attributes["type"]))
    for attr, value in attributes.items():
        if attr != "type":  # Skip 'type' since it's already added
            g.add((HI[instance], HI[attr], Literal(value, datatype=XSD.float if isinstance(value, float) else XSD.boolean)))

# Save the updated ontology
updated_ontology_path = "updated_hi_ontology.ttl"
g.serialize(updated_ontology_path, format="turtle")

print(f"✅ Ontology updated and saved to: {updated_ontology_path}")

✅ Ontology updated and saved to: updated_hi_ontology.ttl


In [24]:
from rdflib import Graph, Namespace, Literal, RDF, RDFS, OWL, XSD, BNode

# Load the existing ontology
ontology_path = "updated_hi_ontology.ttl"
g = Graph()
g.parse(ontology_path, format="turtle")

# Define correct namespace (keeping it consistent)
HI = Namespace("http://www.semanticweb.org/vbr240/ontologies/2022/4/untitled-ontology-51/")
g.bind("hi", HI)

# Define new Classes from the tenth PDF
new_classes = {
    "Value_Aware_Learning": HI.AIConcept,
    "Selective_Classification": HI.Value_Aware_Learning,
    "Active_Learning_Strategy": HI.Value_Aware_Learning,
    "Threshold_Oriented_Sampling": HI.Active_Learning_Strategy,
    "Human_Agent_Interaction": HI.AIConcept,
    "Trust_Aware_DecisionMaking": HI.Human_Agent_Interaction,
}

# Add new classes to the ontology
for cls, parent in new_classes.items():
    g.add((HI[cls], RDF.type, OWL.Class))
    g.add((HI[cls], RDFS.subClassOf, parent))

# Define new Object Properties from the tenth PDF
new_object_properties = {
    "improvesDecisionMaking": (HI.Trust_Aware_DecisionMaking, HI.Human_Agent_Interaction),
    "isUsedIn": (HI.Threshold_Oriented_Sampling, HI.Active_Learning_Strategy),
    "enhancesSelectiveLearning": (HI.Selective_Classification, HI.Value_Aware_Learning),
    "appliesThresholding": (HI.Threshold_Oriented_Sampling, HI.Selective_Classification),
}

# Add Object Properties
for prop, (domain, range_) in new_object_properties.items():
    g.add((HI[prop], RDF.type, OWL.ObjectProperty))
    g.add((HI[prop], RDFS.domain, domain))
    g.add((HI[prop], RDFS.range, range_))

# Define new Datatype Properties from the tenth PDF
new_datatype_properties = {
    "learningEfficiency": (HI.Active_Learning_Strategy, XSD.float),
    "trustLevel": (HI.Trust_Aware_DecisionMaking, XSD.float),
    "classificationConfidence": (HI.Selective_Classification, XSD.float),
}

# Add Datatype Properties
for prop, (domain, dtype) in new_datatype_properties.items():
    g.add((HI[prop], RDF.type, OWL.DatatypeProperty))
    g.add((HI[prop], RDFS.domain, domain))
    g.add((HI[prop], RDFS.range, dtype))

# Define minimum cardinality constraints using a blank node
restriction_node = BNode()
g.add((restriction_node, RDF.type, OWL.Restriction))
g.add((restriction_node, OWL.onProperty, HI.enhancesSelectiveLearning))
g.add((restriction_node, OWL.minCardinality, Literal(1, datatype=XSD.nonNegativeInteger)))

# Attach the restriction to the Selective_Classification class
g.add((HI.Selective_Classification, RDFS.subClassOf, restriction_node))

# Define new Instances related to tenth PDF concepts
new_instances = {
    "Threshold_Sampling_Model": {
        "type": HI.Threshold_Oriented_Sampling,
        "learningEfficiency": 0.89
    },
    "Human_Agent_Trust_Model": {
        "type": HI.Trust_Aware_DecisionMaking,
        "trustLevel": 0.92
    },
    "Selective_Learning_System": {
        "type": HI.Selective_Classification,
        "classificationConfidence": 0.87
    },
}

# Add instances to the ontology
for instance, attributes in new_instances.items():
    g.add((HI[instance], RDF.type, attributes["type"]))
    for attr, value in attributes.items():
        if attr != "type":  # Skip 'type' since it's already added
            g.add((HI[instance], HI[attr], Literal(value, datatype=XSD.float if isinstance(value, float) else XSD.boolean)))

# Save the updated ontology
updated_ontology_path = "updated_hi_ontology.ttl"
g.serialize(updated_ontology_path, format="turtle")

print(f"✅ Ontology updated and saved to: {updated_ontology_path}")

✅ Ontology updated and saved to: updated_hi_ontology.ttl
